# Transfer Learning

## [Xception Introduction](#Xception)

## [Dataset Info](#Dataset)

## Four Major Scenarios
1. [similar dataset, very little data: Use linear classifier on top layer](#Scenario-1:-use-linear-classifier-on-top-layer)
2. [similar dataset, a lot of data: Finetune a few layers](#Scenario-2:-fintune-a-few-layers)
3. [different dataset, very little data: Try linear classifier from different stages](#Scenario-3:-try-linear-classifier-from-different-stages)
4. [different dataset, a lot of data: Finetuen a large number of layers](#Scenario-4:-fintune-a-few-layers)

### [Load pretrained model and change output layer](#Load-pretrained-model-and-change-output-layer)

In [1]:
import os
import sys
import keras
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k

Using TensorFlow backend.


### Xception
extreme inception

[Xception: Deep Learning with Depthwise Separable Convolutions](https://arxiv.org/abs/1610.02357)

Chollet, F. (2016). Xception: Deep Learning with Depthwise Separable Convolutions. arXiv preprint arXiv:1610.02357.
ISO 690

François Chollet: Author of Keras, Google Artificial Intelligence Researcher

ImageNet Top-5 Accuracy: 0.945

keras.applications.xception.Xception(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

[Keras-Xception](https://keras.io/applications/#xception)

![xception structue](https://www.pyimagesearch.com/wp-content/uploads/2017/03/imagenet_xception_flow.png)



In [2]:
xception_model = Xception(input_shape = (128,128,3), weights='imagenet')
print(xception_model.summary())
for ind, layer in enumerate(xception_model.layers):
    print(ind, layer.name) 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

### Dataset
Dataset: [Kaggle - Dogs vs. Cast](https://www.kaggle.com/c/dogs-vs-cats)

Train: dogs 0-999, cats 0-999 from Kaggle train data

Validation: dogs 1000-1400, cats 1000-1400 from Kaggle train data

In [3]:
nb_classes = 2
img_width, img_height = 128, 128
batch_size = 64
epochs = 10
transformation_ratio = 0.05

train_path = './dataset/train/'
preprocess_path = './dataset/preprocess/'
validation_path = './dataset/validation/'
best_model_path = './best_model.model'

# Prepare data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 10,
                                   shear_range = transformation_ratio,
                                   zoom_range = transformation_ratio,
                                   horizontal_flip = True,
                                   vertical_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size = (img_width, img_height),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(validation_path,
                                                              target_size = (img_width, img_height),
                                                              batch_size = batch_size,
                                                              class_mode = 'categorical')

callbacks_list = [ModelCheckpoint(best_model_path, monitor='val_acc', verbose=1, save_best_only=True),
                  EarlyStopping(monitor='val_acc', patience=3, verbose=0)]

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [ ]:
#Generate preprocessing data


if not os.path.exists('./dataset/link/'):
    os.path.mkdirs('./dataset/link/')

if not os.path.exists('./dataset/link/dogs'):
    os.symlink('./dataset/train/dogs', './dataset/link/dogs')
    
if not os.path.exists('./dataset/link/cats'):
    os.symlink('./dataset/train/cats', './dataset/link/cats')
    
if not os.path.exists('./dataset/preprocess/dogs/'):
    os.path.mkdirs('./dataset/preprocess/dogs/')
    
if not os.path.exists('./dataset/preprocess/cats/'):
    os.path.mkdirs('./dataset/preprocess/cats/')

preprocess_datagen = ImageDataGenerator(rotation_range = 90,
                                        horizontal_flip = True,
                                        vertical_flip = True)

if len(os.listdir('./dataset/preprocess/dogs/')) == 0:
    i=0
    for batch in preprocess_datagen.flow_from_directory('./dataset/link/dogs/',
                                                        save_to_dir = './dataset/preprocess/dogs/',
                                                        target_size = (img_width, img_height),
                                                        batch_size = 1000,
                                                        follow_links = True,
                                                        class_mode = None):
        i+=1
        if i >= 10:
            break
            
if len(os.listdir('./dataset/preprocess/cats/')) == 0:
    i=0
    for batch in preprocess_datagen.flow_from_directory('./dataset/link/cats/',
                                                        save_to_dir = './dataset/preprocess/cats/',
                                                        target_size = (img_width, img_height),
                                                        batch_size = 1000,
                                                        follow_links = True,
                                                        class_mode = None):
        i+=1
        if i >= 10:
            break


### Scenario 1: use linear classifier on top layer
Extract features with pretrained model

In [4]:
#Scenario 1: use linear classifier on top layer

def model_1(input_shape):
    
    base_xception_model = Xception(input_shape = input_shape, weights = 'imagenet', include_top = False, pooling='max')
    
    return base_xception_model

model = model_1((img_width, img_height, 3))

preprocess_train_generator = validation_datagen.flow_from_directory(preprocess_path,
                                                                    shuffle = False,
                                                                    target_size = (img_width, img_height),
                                                                    batch_size = batch_size,
                                                                    class_mode = 'categorical')

train_x = model.predict_generator(preprocess_train_generator)
train_y = preprocess_train_generator.classes

preprocess_val_generator = validation_datagen.flow_from_directory(validation_path,
                                                                  shuffle = False,
                                                                  target_size = (img_width, img_height),
                                                                  batch_size = batch_size,
                                                                  class_mode = 'categorical')
val_x = model.predict_generator(preprocess_val_generator)
val_y = validation_generator.classes

print('trian_x shape', train_x.shape)
print('val_x shape', val_x.shape)


Found 20000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.
('trian_x shape', (20000, 2048))
('val_x shape', (802, 2048))


### Scenario 2: fintune a few layers
Finetune last block of xception

In [5]:
#Scenario 2: fintune a few layers
def model_2(input_shape, output_shape):
    
    xception_last_block_layer_index = 126
    
    base_xception_model = Xception(input_shape = input_shape, weights = 'imagenet', include_top = False)
    hid = base_xception_model.output
    hid = GlobalMaxPooling2D()(hid)
    out = Dense(output_shape, activation='softmax')(hid)
    
    for layer in base_xception_model.layers[:xception_last_block_layer_index]:
        layer.trainable = False

    #layer trainable defalut is True
    for layer in base_xception_model.layers[xception_last_block_layer_index:]:
        layer.trainable = True

    model = Model( inputs = base_xception_model.input, outputs = out)
    model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = model_2((img_width, img_height, 3), nb_classes)
model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=callbacks_list)


Epoch 1/10
32/32 [==============================] - 14s 437ms/step - loss: 0.4716 - acc: 0.7631 - val_loss: 0.1848 - val_acc: 0.9239
Epoch 2/10
32/32 [==============================] - 7s 231ms/step - loss: 0.3329 - acc: 0.8554 - val_loss: 0.1759 - val_acc: 0.9289
Epoch 3/10
32/32 [==============================] - 8s 244ms/step - loss: 0.2748 - acc: 0.8795 - val_loss: 0.2083 - val_acc: 0.9227
Epoch 4/10
32/32 [==============================] - 7s 231ms/step - loss: 0.2440 - acc: 0.8985 - val_loss: 0.2219 - val_acc: 0.9190
Epoch 5/10
32/32 [==============================] - 8s 244ms/step - loss: 0.2118 - acc: 0.9121 - val_loss: 0.2544 - val_acc: 0.9102


### Scenario 3: try linear classifier from different stages
Extract features with pretrained model from different stages

In [7]:
#Scenario 3: fintune a few layers
def model_3(input_shape, block_index):
    
    block_layers = range(15,135,10)
    
    base_xception_model = Xception(input_shape = input_shape, weights = 'imagenet', include_top = False)
    base_xception_model.layers = base_xception_model.layers[:block_layers[block_index]]
    base_xception_model.outputs = [base_xception_model.layers[-1].output]
    base_xception_model.layers[-1].outbound_node = []
    
    hid = base_xception_model.layers[-1].output
    out = GlobalMaxPooling2D()(hid)

    model = Model( inputs = base_xception_model.input, outputs = out)
    model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = model_3((img_width, img_height, 3), 5)
model.summary()

preprocess_train_generator = validation_datagen.flow_from_directory(preprocess_path,
                                                                    shuffle = False,
                                                                    target_size = (img_width, img_height),
                                                                    batch_size = batch_size,
                                                                    class_mode = 'categorical')

train_x = model.predict_generator(preprocess_train_generator)
train_y = preprocess_train_generator.classes

preprocess_val_generator = validation_datagen.flow_from_directory(validation_path,
                                                                  shuffle = False,
                                                                  target_size = (img_width, img_height),
                                                                  batch_size = batch_size,
                                                                  class_mode = 'categorical')
val_x = model.predict_generator(preprocess_val_generator)
val_y = validation_generator.classes

print('trian_x shape', train_x.shape)
print('val_x shape', val_x.shape)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Found 802 images belonging to 2 classes.
('trian_x shape', (20000, 728))
('val_x shape', (802, 728))


### Scenario 4: fintune a few layers
Finetune a large number of layers

In [8]:
#Scenario 4: finetune all layers
def model_4(input_shape, output_shape):
    
    base_xception_model = Xception(input_shape = input_shape, weights = 'imagenet', include_top = False)
    hid = base_xception_model.output
    hid = GlobalMaxPooling2D()(hid)
    out = Dense(output_shape, activation='softmax')(hid)
    
    #layer trainable defalut is True
    for layer in base_xception_model.layers:
        layer.trainable = True

    model = Model( inputs = base_xception_model.input, outputs = out)
    model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = model_4((img_width, img_height, 3), nb_classes)
model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=callbacks_list)

Epoch 1/10
32/32 [==============================] - 31s 981ms/step - loss: 0.4301 - acc: 0.8077 - val_loss: 0.1771 - val_acc: 0.9426
Epoch 2/10
32/32 [==============================] - 14s 443ms/step - loss: 0.2021 - acc: 0.9204 - val_loss: 0.2478 - val_acc: 0.9451
Epoch 3/10
32/32 [==============================] - 13s 416ms/step - loss: 0.1242 - acc: 0.9434 - val_loss: 0.2958 - val_acc: 0.9377
Epoch 4/10
32/32 [==============================] - 13s 413ms/step - loss: 0.1053 - acc: 0.9644 - val_loss: 0.3898 - val_acc: 0.9040
Epoch 5/10
32/32 [==============================] - 14s 441ms/step - loss: 0.0626 - acc: 0.9790 - val_loss: 0.1764 - val_acc: 0.9539
Epoch 6/10
32/32 [==============================] - 14s 448ms/step - loss: 0.0575 - acc: 0.9805 - val_loss: 0.1815 - val_acc: 0.9589
Epoch 7/10
32/32 [==============================] - 14s 442ms/step - loss: 0.0272 - acc: 0.9927 - val_loss: 0.1633 - val_acc: 0.9613
Epoch 8/10
32/32 [==============================] - 13s 412ms/step - 

### Load pretrained model and change output layer

In [9]:
from keras.models import load_model

def custom_model(output_shape, model_path):
    pretrained_model = load_model('./best_model.model')

    pretrained_model.layers.pop()
    pretrained_model.outputs = [pretrained_model.layers[-1].output]
    pretrained_model.layers[-1].outbound_node = []
    
    hid = pretrained_model.output
    out = Dense(output_shape, activation='softmax')(hid)
    
    model = Model( inputs = pretrained_model.input, outputs= out)
    model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

model = custom_model(2, best_model_path)
model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=callbacks_list)

Epoch 1/10
32/32 [==============================] - 20s 626ms/step - loss: 0.9523 - acc: 0.5000 - val_loss: 0.9417 - val_acc: 0.5000
Epoch 2/10
32/32 [==============================] - 14s 426ms/step - loss: 0.9270 - acc: 0.5014 - val_loss: 0.9191 - val_acc: 0.5000
Epoch 3/10
32/32 [==============================] - 13s 415ms/step - loss: 0.9075 - acc: 0.4986 - val_loss: 0.8962 - val_acc: 0.5000
Epoch 4/10
32/32 [==============================] - 13s 417ms/step - loss: 0.8792 - acc: 0.5014 - val_loss: 0.8729 - val_acc: 0.5000
Epoch 5/10
32/32 [==============================] - 13s 418ms/step - loss: 0.8495 - acc: 0.5057 - val_loss: 0.8507 - val_acc: 0.5000
Epoch 6/10
32/32 [==============================] - 13s 414ms/step - loss: 0.8350 - acc: 0.4986 - val_loss: 0.8264 - val_acc: 0.5000
Epoch 7/10
32/32 [==============================] - 13s 421ms/step - loss: 0.8110 - acc: 0.4986 - val_loss: 0.8035 - val_acc: 0.5000
Epoch 8/10
32/32 [==============================] - 14s 423ms/step - 